[Долгая краткосрочная память](https://neerc.ifmo.ru/wiki/index.php?title=%D0%94%D0%BE%D0%BB%D0%B3%D0%B0%D1%8F_%D0%BA%D1%80%D0%B0%D1%82%D0%BA%D0%BE%D1%81%D1%80%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%BF%D0%B0%D0%BC%D1%8F%D1%82%D1%8C)

**keras**

In [1]:
# Импорты
import numpy as np
import keras.backend as K
from keras.preprocessing import sequence
from keras.utils.data_utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Dropout
from keras.layers import LSTM
from keras.datasets import imdb

In [2]:
def f1(y_true, y_pred):
    
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [3]:
# Устанавливаем seed для обеспечения повторяемости результатов
np.random.seed(42)

# Указываем количество слов из частотного словаря, которое будет использоваться (отсортированы по частоте использования)
max_features = 5000

In [4]:
# Загружаем данные (датасет IMDB содержит 25000 рецензий на фильмы с правильным ответом для обучения и 25000 рецензий на фильмы с правильным ответом для тестирования)
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words = max_features)

In [5]:
# Устанавливаем максимальную длину рецензий в словах, чтобы они все были одной длины
maxlen = 80

In [6]:
# Заполняем короткие рецензии пробелами, а длинные обрезаем
X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

In [7]:
# Создаем модель последовательной сети
model = Sequential()
# Добавляем слой для векторного представления слов (5000 слов, каждое представлено вектором из 32 чисел, отключаем входной сигнал с вероятностью 20% для предотвращения переобучения)
model.add(Embedding(max_features, 32))
model.add(Dropout(0.2))
# Добавляем слой долго-краткосрочной памяти (100 элементов для долговременного хранения информации, отключаем входной сигнал с вероятностью 20%, отключаем рекуррентный сигнал с вероятностью 20%)
model.add(LSTM(100))
model.add(Dropout(0.2))
# Добавляем полносвязный слой из 1 элемента для классификации, в качестве функции активации будем использовать сигмоидальную функцию
model.add(Dense(1, activation = 'sigmoid'))

# Компилируем модель нейронной сети
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy', f1]
)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          160000    
                                                                 
 dropout (Dropout)           (None, None, 32)          0         
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Обучаем нейронную сеть (данные для обучения, ответы к данным для обучения, количество рецензий после анализа которого будут изменены веса, число эпох обучения, тестовые данные, показывать progress bar или нет)
model.fit(
    X_train, 
    y_train, 
    batch_size = 64,
    epochs = 7,
    validation_data = (X_test, y_test),
    verbose = 1
)

Epoch 1/7
391/391 [==============================] - 39s 96ms/step - loss: 0.4646 - accuracy: 0.7689 - f1: 0.7701 - val_loss: 0.3723 - val_accuracy: 0.8302 - val_f1: 0.8401
Epoch 2/7
391/391 [==============================] - 37s 96ms/step - loss: 0.3248 - accuracy: 0.8637 - f1: 0.8627 - val_loss: 0.3920 - val_accuracy: 0.8222 - val_f1: 0.8334
Epoch 3/7
391/391 [==============================] - 38s 98ms/step - loss: 0.3009 - accuracy: 0.8750 - f1: 0.8744 - val_loss: 0.3797 - val_accuracy: 0.8313 - val_f1: 0.8189
Epoch 4/7
391/391 [==============================] - 35s 90ms/step - loss: 0.2633 - accuracy: 0.8923 - f1: 0.8907 - val_loss: 0.3816 - val_accuracy: 0.8292 - val_f1: 0.8349
Epoch 5/7
391/391 [==============================] - 35s 90ms/step - loss: 0.2376 - accuracy: 0.9055 - f1: 0.9042 - val_loss: 0.4025 - val_accuracy: 0.8314 - val_f1: 0.8254
Epoch 6/7
391/391 [==============================] - 35s 89ms/step - loss: 0.2120 - accuracy: 0.9170 - f1: 0.9161 - val_loss: 0.4733 - 

In [10]:
# Проверяем качество обучения на тестовых данных (если есть данные, которые не участвовали в обучении, лучше использовать их, но в нашем случае таковых нет)
scores = model.evaluate(X_test, y_test, batch_size = 64)

print('Точность на тестовых данных: %.2f%%' % (scores[1] * 100))
print('F1 на тестовых данных: %.2f%%' % (scores[2] * 100))

391/391 [==============================] - 8s 20ms/step - loss: 0.5389 - accuracy: 0.8220 - f1: 0.8144
Точность на тестовых данных: 82.20%
F1 на тестовых данных: 81.44%


**tensorflow**

[Tensorflow LSTM](https://www.educba.com/tensorflow-lstm/)

In [59]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [60]:
# Configurations related to the model
extra_metric_param = ['accuracy']
size_of_single_batch = 128
output_of_dims = 15
function_used_for_Loss= BinaryCrossentropy()
length_of_maximum_sequence = 300
distinct_words_number = 5000
epochs_count = 5
optimizer = Adam()
split_for_validation = 0.20
Mode_Verbose = 1

In [61]:
# Execution should be in eager mode, so disable it
tf.compat.v1.disable_eager_execution()

In [62]:
# Data set should be loaded first
(x_train, y_train), (test_for_X_coordinate, test_for_Y_coordinate) = imdb.load_data(num_words=distinct_words_number)

print(x_train.shape)
print(test_for_X_coordinate.shape)

(25000,)
(25000,)


In [63]:
# Sequences should have proper padding between them
padded_inputs = pad_sequences(
    x_train, 
    maxlen=length_of_maximum_sequence, 
    value = 0.0
) # 0.0 because it corresponds with <PAD>

padded_inputs_test = pad_sequences(
    test_for_X_coordinate, 
    maxlen=length_of_maximum_sequence, 
    value = 0.0
) # 0.0 because it corresponds with <PAD>

In [65]:
# The keras model that will be used should be defined properly
model = Sequential()

model.add(Embedding(distinct_words_number, output_of_dims, input_length=length_of_maximum_sequence))
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))

# Model is compiled
model.compile(optimizer=optimizer, loss=BinaryCrossentropy(), metrics=extra_metric_param)

In [66]:
# Providing brief summary about the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 15)           75000     
                                                                 
 lstm_2 (LSTM)               (None, 10)                1040      
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 76,051
Trainable params: 76,051
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Model is being trained
history = model.fit(
    padded_inputs, 
    y_train, 
    batch_size=size_of_single_batch, 
    epochs=epochs_count, 
    verbose=Mode_Verbose, 
    validation_split=split_for_validation
)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - ETA: 0s - loss: 0.4589 - acc: 0.8025

C:\Python310\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


20000/20000 [==============================] - 13s 637us/sample - loss: 0.4589 - acc: 0.8025 - val_loss: 0.3648 - val_acc: 0.8544
Epoch 2/5
20000/20000 [==============================] - 13s 645us/sample - loss: 0.2985 - acc: 0.8867 - val_loss: 0.3164 - val_acc: 0.8724
Epoch 3/5
20000/20000 [==============================] - 13s 647us/sample - loss: 0.2394 - acc: 0.9123 - val_loss: 0.3140 - val_acc: 0.8746
Epoch 4/5
20000/20000 [==============================] - 13s 651us/sample - loss: 0.2101 - acc: 0.9259 - val_loss: 0.3094 - val_acc: 0.8732
Epoch 5/5
20000/20000 [==============================] - 13s 654us/sample - loss: 0.1832 - acc: 0.9381 - val_loss: 0.3485 - val_acc: 0.8660


In [70]:
# Model should be tested after the completion of the training
result_of_testing = model.evaluate(padded_inputs_test, test_for_Y_coordinate, verbose=False)

print(f'Results of the final test in case of Loss: {result_of_testing[0]} and the accuracy of model is : {result_of_testing[1]}')

Results of the final test in case of Loss: 0.36116048718452454 and the accuracy of model is : 0.8590400218963623
